In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# 指定模型ID
# https://huggingface.co/Qwen/collections
# model_id = "Qwen/Qwen1.5-0.5B-Chat" # ❌
# model_id = "Qwen/Qwen3-Coder-30B-A3B-Instruct" # crash
# model_id = "Qwen/Qwen2.5-Math-7B" # ✅️
model_id = "Qwen/Qwen3-0.6B" # ✅️

# 设置设备，优先使用GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# 加载分词器
tokenizer = AutoTokenizer.from_pretrained(model_id)

# 加载模型，并将其移动到指定设备
model = AutoModelForCausalLM.from_pretrained(model_id).to(device)

print("模型和分词器加载完成！")

# 准备对话输入
# prompt = "你好，请介绍你自己。"
# 正确的计算应该是（48+12）/（80+15）= 60/95=12/19≈0.6316，也就是约63.16%
prompt = "一个篮球队在一个赛季的80场比赛中赢了60%。在接下来的赛季中，他们打了15场比赛，赢了12场。两个赛季的总胜率是多少？"
# prompt = "一个篮球队在一个赛季的80场比赛中赢了60%。在接下来的赛季中，他们打了15场比赛，赢了12场。两个赛季的总胜率是多少？请逐步思考解答。"
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
]

# 使用分词器的模板格式化输入
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
# print("格式化后的输入文本:")
# print(text)

# 编码输入文本
model_inputs = tokenizer([text], return_tensors="pt").to(device)
print("编码后的输入文本:")
print(model_inputs)

# 使用模型生成回答
# max_new_tokens 控制了模型最多能生成多少个新的Token
generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=8192
)
# print("生成的 Token IDs:")
# print(generated_ids)

# 将生成的 Token ID 截取掉输入部分
# 这样我们只解码模型新生成的部分
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
# print("截取后的生成 Token IDs:")
# print(generated_ids)

# 解码生成的 Token ID
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print("\n模型的回答:")
print(response)

# 用完显式释放
# del model, tokenizer, model_inputs, generated_ids
# torch.cuda.empty_cache()
# torch.cuda.ipc_collect()


Using device: cuda
模型和分词器加载完成！
格式化后的输入文本:
<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
一个篮球队在一个赛季的80场比赛中赢了60%。在接下来的赛季中，他们打了15场比赛，赢了12场。两个赛季的总胜率是多少？<|im_end|>
<|im_start|>assistant

编码后的输入文本:
{'input_ids': tensor([[151644,   8948,    198,   2610,    525,    264,  10950,  17847,     13,
         151645,    198, 151644,    872,    198,  46944, 100926, 102151, 106568,
         100581,   9370,     23,     15, 101676,  15946,  99959,  34187,     21,
             15,      4,   1773,  18493, 104326,   9370, 100581,  15946,   3837,
          99650, 106677,     16,     20, 101676,   3837,  99959,  34187,     16,
             17,  82224,   1773, 100369, 100581,   9370,  59743,  99813,  95355,
         111558,  11319, 151645,    198, 151644,  77091,    198]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       

In [ ]:
# 所有模型列表都存储在 Hugging Face Hub 上，可以使用以下代码列出模型。（太慢!!!)
# from huggingface_hub import list_models

# models = list(list_models())
# print(f"Total models found: {len(models)}")
# list_models(filter="text-generation")
# list_models(filter="pytorch")
# models = list(list_models(filter="transformers"))
# print(f"Transformers models found: {len(models)}")

# for m in models:
#     print(m.modelId)
